In [4]:
#import sys
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install opencv-python

'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.
'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.
'{sys.executable}' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
import numpy as np
from sklearn.model_selection import train_test_split
import os
import cv2

In [15]:
# Import the data and check file count
data = r'Train'

files = []
for _, _, files in os.walk(data):
    break
print(f'Total number of files: {len(files)}')


Total number of files: 971


In [16]:
Y = np.zeros((len(files), 3))

for i, f in enumerate(files):
    if 'lower' in f:
        Y[i, 0] = 1  # Label for Lower class
    elif 'middle' in f:
        Y[i, 1] = 1  # Label for Middle class
    elif 'upper' in f:
        Y[i, 2] = 1  # Label for Upper Class

files = [os.path.join(data, f) for f in files]


In [17]:
X = []
for file in files:
    img = cv2.imread(file)  # Read the image
    img = cv2.resize(img, (150, 150))  # Resize the image to 150x150, tried higher values like 256x256 and even 512x512 and got worse results
    X.append(img)

X = np.array(X, dtype='float32') / 255.0

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=38)

In [19]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, Y_test)

# Print the test accuracy
print(f"Test accuracy: {test_acc:.6f}")

Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 233ms/step - accuracy: 0.3774 - loss: 4.5681 - val_accuracy: 0.4231 - val_loss: 1.1221
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - accuracy: 0.4376 - loss: 1.0551 - val_accuracy: 0.4295 - val_loss: 1.0354
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 214ms/step - accuracy: 0.5136 - loss: 1.0004 - val_accuracy: 0.4231 - val_loss: 1.0240
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 204ms/step - accuracy: 0.4532 - loss: 0.9759 - val_accuracy: 0.4359 - val_loss: 0.9931
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.6046 - loss: 0.9138 - val_accuracy: 0.4679 - val_loss: 0.9880
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4468 - loss: 1.0434
Test accuracy: 0.446154


In [21]:
resbase = applications.ResNet50V2(weights = 'imagenet', include_top=False, input_shape=(150,150,3))
resbase.trainable = False

resModel = models.Sequential([
    resbase,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation= 'relu' ),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

resModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [23]:
resModel.fit(X_train, Y_train, epochs = 5, batch_size=64, validation_split=0.2)

Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 13s 878ms/step - accuracy: 0.4617 - loss: 1.7231 - val_accuracy: 0.4551 - val_loss: 1.0819
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 834ms/step - accuracy: 0.5271 - loss: 1.0362 - val_accuracy: 0.5577 - val_loss: 0.9721
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 933ms/step - accuracy: 0.6212 - loss: 0.7767 - val_accuracy: 0.5705 - val_loss: 0.9484
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.7309 - loss: 0.6460 - val_accuracy: 0.5513 - val_loss: 0.9550
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 927ms/step - accuracy: 0.7503 - loss: 0.5918 - val_accuracy: 0.5962 - val_loss: 0.9522


In [25]:
test_loss, test_acc = resModel.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc:.6f}")

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 345ms/step - accuracy: 0.5123 - loss: 1.0792
Test accuracy: 0.512821
